<a href="https://colab.research.google.com/github/uniquing/API-Connection/blob/main/WALMART_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import pandas as pd
import re
import time
import datetime
import base64
from requests.auth import HTTPBasicAuth

### Get Token First


In [ ]:
# Use to authorization(clientId:clientsecret)
client_id="your id"
client_sec="your secert"
usrPass=client_id+":"+client_sec
# Transform the code for authoriztion
encoded_u = base64.b64encode(usrPass.encode()).decode()

In [ ]:
# The header pramas from API document
HeaderParameters={
    'Authorization' : "Basic %s" % encoded_u, #Should be the kind of 'Basic blablabla...' so use %s to concatenate
    'Content-Type': 'application/x-www-form-urlencoded',
    'Accept': 'application/json',
    'WM_SVC.NAME': 'Walmart Marketplace',
    'WM_QOS.CORRELATION_ID': '123456abcdef', #Could use any GUID or UUID
    'WM_SVC.VERSION': '1.0.0'
}

In [ ]:
# Get the token to access
url='https://marketplace.walmartapis.com/v3/token'
token_resp=requests.post(url,headers=HeaderParameters,data={"grant_type": "client_credentials"})
token_resp.status_code

200

### Get the Orders (can try another like feed, items etc.)


In [ ]:
# The header use to access the orders database
common_headers={
    'Authorization': "Basic %s" % encoded_u,
    'WM_SVC.NAME': 'Walmart Marketplace',
    'WM_SEC.ACCESS_TOKEN': token_resp.json()['access_token'],
    'WM_QOS.CORRELATION_ID': '123456abcdef', #Could use any GUID or UUID
    'Accept': 'application/json',
    'Host': 'marketplace.walmartapis.com',
    #'limit': '199'
    #'createdStartDate':'2019-10-18',
    #'createdEndDate':'2020-4-18'
    }


In [ ]:
#Try to have start date and limit for 200 orders
order_url = "https://marketplace.walmartapis.com/v3/orders?createdStartDate=2018-01-25&limit=200"

order_resp=requests.get(order_url, headers=common_headers)
order_data=order_resp.json()
order_resp.status_code

200

In [ ]:
df = pd.DataFrame()
for i in list(range(0,len(order_data['list']['elements']['order']))):

  order_dict={
      'Email_id': order_data['list']['elements']['order'][i]['customerEmailId'],
      'customerOrderId': order_data['list']['elements']['order'][i]['customerOrderId'],
      'orderDate': time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(order_data['list']['elements']['order'][i]['orderDate']/1000)),
      'chargeAmount': order_data['list']['elements']['order'][i]['orderLines']['orderLine'][0]['charges']['charge'][0]['chargeAmount']['amount'],
      'currency': order_data['list']['elements']['order'][i]['orderLines']['orderLine'][0]['charges']['charge'][0]['chargeAmount']['currency'],
      'chargeName': order_data['list']['elements']['order'][i]['orderLines']['orderLine'][0]['charges']['charge'][0]['chargeName'],
      'chargeType': order_data['list']['elements']['order'][i]['orderLines']['orderLine'][0]['charges']['charge'][0]['chargeType'],
      'fulfillmentOption': order_data['list']['elements']['order'][i]['orderLines']['orderLine'][0]['fulfillment']['fulfillmentOption'],
      'pickUpBy': order_data['list']['elements']['order'][i]['orderLines']['orderLine'][0]['fulfillment']['pickUpBy'],
      'pickUpDateTime': time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(order_data['list']['elements']['order'][i]['orderLines']['orderLine'][0]['fulfillment']['pickUpDateTime']/1000)),
      'shipMethod': order_data['list']['elements']['order'][i]['orderLines']['orderLine'][0]['fulfillment']['shipMethod'],
      'shippingProgramType': order_data['list']['elements']['order'][i]['orderLines']['orderLine'][0]['fulfillment']['shippingProgramType'],
      'storeId': order_data['list']['elements']['order'][i]['orderLines']['orderLine'][0]['fulfillment']['storeId'],
      'productName': order_data['list']['elements']['order'][i]['orderLines']['orderLine'][0]['item']['productName'],
      'sku': order_data['list']['elements']['order'][i]['orderLines']['orderLine'][0]['item']['sku'],
      'cancellationReason': order_data['list']['elements']['order'][i]['orderLines']['orderLine'][0]['orderLineStatuses']['orderLineStatus'][0]['cancellationReason'],
      'refund': order_data['list']['elements']['order'][i]['orderLines']['orderLine'][0]['refund'],
      'order_statusdate': time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(order_data['list']['elements']['order'][i]['orderLines']['orderLine'][0]['statusDate']/1000)),
      'purchaseOrderId': order_data['list']['elements']['order'][i]['purchaseOrderId'],
      'shiptype': order_data['list']['elements']['order'][i]['shipNode']['type'],
      'postalCode': order_data['list']['elements']['order'][i]['shippingInfo']['postalAddress']['postalCode'],
      'shipping_city': order_data['list']['elements']['order'][i]['shippingInfo']['postalAddress']['city'],
      'state': order_data['list']['elements']['order'][i]['shippingInfo']['postalAddress']['state'],
      'shipping_country': order_data['list']['elements']['order'][i]['shippingInfo']['postalAddress']['country']
    }
  df_ = pd.DataFrame(order_dict, index=[i])
  df = pd.concat([df,df_], ignore_index=True)

df

In [ ]:
df.to_csv('order.csv')